In [1]:
import pandas as pd

In [2]:
df_bar = pd.read_csv('Street_Tree_List-2022-01-30_RAW.csv')

/var/folders/w2/02kgc33150gg540z2kvhmgzm0000gn/T/ipykernel_61692/2505361486.py:1: DtypeWarning: Columns (1,2,3,5,6,7,9) have mixed types. Specify dtype option on import or set low_memory=False.
  df_bar = pd.read_csv('Street_Tree_List-2022-01-30_RAW.csv')


In [3]:
df_bar = df_bar.dropna(subset=['PlantDate'])

In [4]:
df_bar = df_bar[df_bar['qCaretaker'].isin(['Private', 'DPW'])]

In [5]:
df_bar['Year'] = pd.to_datetime(df_bar['PlantDate'], format='%m/%d/%y %H:%M').dt.year

In [6]:
df_bar = df_bar.dropna(subset=['Year'])

In [7]:
df_bar['Year'] = df_bar['Year'].astype(int)

In [8]:
df_bar = df_bar[df_bar['Year'] <= 2021]

In [9]:
annual_summary = df_bar.groupby('Year').agg(
    Total_Trees=pd.NamedAgg(column='TreeID', aggfunc='count'),
    Private_Trees=pd.NamedAgg(column='qCaretaker', aggfunc=lambda x: (x == 'Private').sum()),
    DPW_Trees=pd.NamedAgg(column='qCaretaker', aggfunc=lambda x: (x == 'DPW').sum())
).reset_index()

In [10]:
print(annual_summary)

    Year  Total_Trees  Private_Trees  DPW_Trees
0   1969           87             87          0
1   1970           21             21          0
2   1972          409            409          0
3   1973          168            168          0
4   1974          103            103          0
5   1975           20             20          0
6   1976           90             90          0
7   1977           79             79          0
8   1978           61             61          0
9   1979          101            101          0
10  1980          138            136          2
11  1981          113            112          1
12  1982          149            147          2
13  1983          164            163          1
14  1984          112            112          0
15  1985          168            168          0
16  1986          201            200          1
17  1987          214            214          0
18  1988          140            139          1
19  1989          181            170    

In [11]:
annual_summary.to_csv('annual_summary.csv', index=False)

In [12]:
df_pie = pd.read_csv('Street_Tree_List-2022-01-30_RAW.csv')

/var/folders/w2/02kgc33150gg540z2kvhmgzm0000gn/T/ipykernel_61692/2172559333.py:1: DtypeWarning: Columns (1,2,3,5,6,7,9) have mixed types. Specify dtype option on import or set low_memory=False.
  df_pie = pd.read_csv('Street_Tree_List-2022-01-30_RAW.csv')


In [13]:
df_pie.dropna(subset=['qSpecies'], inplace=True)

In [14]:
df_pie = df_pie[df_pie['qCaretaker'].isin(['Private', 'DPW'])]

In [15]:
df_pie['qSpecies'] = df_pie['qSpecies'].apply(lambda x: x.split(' :: ')[-1].strip())

In [16]:
df_pie = df_pie[~df_pie['qSpecies'].str.contains("Tree\(s\) ::", na=False)]

In [17]:
df_pie = df_pie.groupby(['qCaretaker', 'qSpecies']).size().reset_index(name='Count')

In [18]:
print(df_pie)

    qCaretaker               qSpecies  Count
0          DPW                     ::     13
1          DPW                :: Tree      1
2          DPW             Acacia Spp     12
3          DPW  Acacia: Silver Wattle      1
4          DPW                 Almond      4
..         ...                    ...    ...
694    Private             Yellow Box      1
695    Private             Yellow Gum     14
696    Private               Yew Pine     47
697    Private         Yoshino Cherry     72
698    Private                  Yucca      3

[699 rows x 3 columns]


In [19]:
df_pie.to_csv('caretaker_and_species.csv', index=False)